# Оптимизация Сети Добычи
модель для предсказания оптимальных связей между нефтяными и газовыми скважинами и центрами сбора или переработки. Здесь узлы - это скважины, а рёбра - логистические или технологические связи. Каждому месторождению соответствует вектор атрибутов: название, страна, регион месторождения, тектонический режим, тип углеводорода, структурные свойства, литология, литологический период, проницаемость, долгота и широта, валовая прибыль.

In [33]:
import torch_geometric.transforms as T
from stable_gnn.model_link_predict import ModelLinkPrediction
from stable_gnn.graph import Graph

## Загружаем данные исходного графа

In [34]:
root = '../data_validation/'
name = 'oil_fields'
adjust_flag = False
dataset = Graph(root=root + str(name), name=name, transform=T.NormalizeFeatures(),adjust_flag=adjust_flag)

Processing...
Done!


## Обучаем модель link prediction
### Разбиене данных на train test

In [46]:
model = ModelLinkPrediction(number_of_trials=50)
train_edges, train_negative, test_edges, test_negative = model.train_test_edges(dataset)

### Обучаем классификатор на тренировочных ребрах, который выдает 1 если ребро есть и 0 -- если нет

In [47]:
cl = model.train_cl(train_edges,train_negative)

[I 2023-12-13 17:14:54,483] A new study created in memory with name: no-name-cf34933a-e673-4735-97e9-5c4cd15df50b
[I 2023-12-13 17:15:00,561] Trial 0 finished with value: 4.94843864440918 and parameters: {'hidden_layer': 64, 'dropout': 0.0, 'size of network, number of convs': 2, 'lr': 0.006455795076076083, 'out_layer': 64, 'num_negative_samples': 16, 'alpha': 0.8}. Best is trial 0 with value: 4.94843864440918.
[I 2023-12-13 17:15:10,028] Trial 1 finished with value: 4.445287704467773 and parameters: {'hidden_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 3, 'lr': 0.009848055947467238, 'out_layer': 64, 'num_negative_samples': 16, 'alpha': 0.4}. Best is trial 1 with value: 4.445287704467773.
[I 2023-12-13 17:15:20,654] Trial 2 finished with value: 4.561388969421387 and parameters: {'hidden_layer': 32, 'dropout': 0.0, 'size of network, number of convs': 3, 'lr': 0.009228921694007648, 'out_layer': 128, 'num_negative_samples': 6, 'alpha': 0.5}. Best is tria

### Проведем тестирование и узнаем точность модели

In [48]:
print("f1 measure", (model.test(cl,test_edges,test_negative)))

f1 measure 0.8305010893246187


## Загружаем данные и уточняем граф с помощью флага adjust_flag = True

In [49]:
root = '../data_validation/'
name='oil_fields'
adjust_flag = True

dataset = Graph(root=root + str(name), name=name, transform=T.NormalizeFeatures(), adjust_flag=adjust_flag)

Processing...
Done!


In [50]:
len(dataset[0].x)

396

### Обучаем классификатор на тренировочных ребрах

In [51]:
model = ModelLinkPrediction(number_of_trials=50)
train_edges, train_negative, test_edges, test_negative = model.train_test_edges(dataset)
cl = model.train_cl(train_edges,train_negative)

[I 2023-12-13 17:26:19,290] A new study created in memory with name: no-name-524615ab-2416-46f0-88b0-85aa3f39024d
[I 2023-12-13 17:26:23,044] Trial 0 finished with value: 3.8365793228149414 and parameters: {'hidden_layer': 64, 'dropout': 0.0, 'size of network, number of convs': 3, 'lr': 0.006066350278122684, 'out_layer': 128, 'num_negative_samples': 16, 'alpha': 0.5}. Best is trial 0 with value: 3.8365793228149414.
[I 2023-12-13 17:26:28,573] Trial 1 finished with value: 2.628662347793579 and parameters: {'hidden_layer': 64, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.005182580427943447, 'out_layer': 64, 'num_negative_samples': 6, 'alpha': 0.1}. Best is trial 1 with value: 2.628662347793579.
[I 2023-12-13 17:26:36,475] Trial 2 finished with value: 2.884071111679077 and parameters: {'hidden_layer': 128, 'dropout': 0.0, 'size of network, number of convs': 2, 'lr': 0.005040593204677637, 'out_layer': 32, 'num_negative_samples': 16, 'alpha': 0.2}. Best is trial 1 with val

### Проведем тестирование и узнаем точность модели для уточненного графа

In [52]:
print("f1 measure", (model.test(cl,test_edges,test_negative)))

f1 measure 0.5714285714285714
